# Get data

In [1]:
!pip -q install -U datasets
!pip -q install transformers[torch]
!pip -q install -U accelerate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.1 which is incompatible.
cudf 24.4.1 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.5.0 which

In [2]:
# import pandas as pd
# import numpy as np
# from io import StringIO
# import random
# import matplotlib.pyplot as plt
# import seaborn as sns

from datasets import load_dataset, load_metric
from datasets import Dataset, DatasetDict
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer

# import torch
# import torch.nn as nn
# from torch.nn import functional as F

# from sklearn.model_selection import train_test_split
# import neattext as nt
# import re
# import nltk
# from nltk.corpus import stopwords
# from tokenizers import ByteLevelBPETokenizer


import warnings
warnings.filterwarnings('ignore')

2024-06-14 12:56:43.486435: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 12:56:43.486529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 12:56:43.614377: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
data = load_dataset("databricks/databricks-dolly-15k")

selected_columns = ["instruction", "context"]
data = data["train"].map(lambda example: {column: example[column] for column in selected_columns}, remove_columns=data["train"].column_names)

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [16]:
data

Dataset({
    features: ['instruction', 'context'],
    num_rows: 15011
})

In [4]:
max_len = len(data)
sp1 = int(0.75 * max_len)
sp2 = int(sp1 + (0.5*(max_len-sp1)))

split1 = data.select(range(sp1)).filter(lambda x: len(x['context'])>0)
split2 = data.select(range(sp1, sp2)).filter(lambda x: len(x['context'])>0)
split3 = data.select(range(sp2, max_len)).filter(lambda x: len(x['context'])>0)


my_data = {'train': split1,
          'validation': split2,
          'test': split3}

my_data = DatasetDict(my_data)

Filter:   0%|          | 0/11258 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1876 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1877 [00:00<?, ? examples/s]

# Tune model

In [5]:
device = 'gpu'

model_base = 'google/flan-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_base)
model = AutoModelForSeq2SeqLM.from_pretrained(model_base)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
pipe = pipeline('text2text-generation', model=model_base)
gen_kwargs = {'length_penalty': 0.8, 'num_beams': 8, 'max_length': 128}

ctxt = my_data['test'][10]['context']

query = ctxt
qout = pipe(query, **gen_kwargs)

print(f"Context: {ctxt}", end="\n\n\n")
print(f"Generated: {qout[0]['generated_text']}")

Context: On Wednesday, September 25, 1974, the race started. It began with a run of a 4.8 km (3 mi) loop, followed by biking twice around Fiesta Island for a total of 8.0 km (5 mi). Entrants would then get off the bikes, take their shoes off and run into the water to swim to the mainland. That was followed by running in bare feet, then swimming again along the bay, then one last swim up to the entrance of Fiesta Island, and a final crawl up a steep dirt bank to finish. Most participants were not skilled swimmers, so Johnstone recruited his 13-year-old son to float on his surfboard and act as lifeguard. Some participants took longer than expected, and it began to get dark as they finished their swims. Shanahan recalls they pulled up a few cars and turned on the headlights so the athletes could see. Johnstone and Shanahan were surprised by the large number of entrants (46), mainly coming from local running clubs. Two notable entrants, Judy and John Collins, would four years later found t

In [17]:
ctxt

'István Szalay (22 March 1944 – 1 September 2022) was a Hungarian mathematician and politician. A member of the Hungarian Socialist Party, he served in the National Assembly from 1998 to 2002. Prior to that, he was mayor of Szeged from 1994 to 1998.\n\nSzalay died on 1 September 2022, at the age of 78.'

In [7]:
def feature_engineering(batch):
    encodings = tokenizer(batch['context'], text_target=batch['instruction'],
                         max_length=1024, truncation=True)

    encodings = {'input_ids': encodings['input_ids'],
                'attention_mask': encodings['attention_mask'],
                'labels': encodings['labels']}

    return encodings

In [8]:
def get_training_args(output_dir):

    training_args = TrainingArguments(
        output_dir = output_dir,
        num_train_epochs = 5,
        warmup_steps = 500,
        auto_find_batch_size = True,
        weight_decay = 0.01,
        logging_steps = 10,
        evaluation_strategy = 'steps',
        eval_steps = 500,
        save_steps = 1e6,
        gradient_accumulation_steps = 16,
        push_to_hub = True
    )

    return training_args

In [9]:
data_tensors = my_data.map(feature_engineering, batched=True)
columns = ['input_ids', 'labels', 'attention_mask']
data_tensors.set_format(type='torch', columns=columns)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Map:   0%|          | 0/3342 [00:00<?, ? examples/s]

Map:   0%|          | 0/561 [00:00<?, ? examples/s]

Map:   0%|          | 0/564 [00:00<?, ? examples/s]

In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
training_args = get_training_args('/kaggle/working/flan_t5_base_args')

trainer = Trainer(model=model, args=training_args, tokenizer=tokenizer, data_collator=data_collator,
                 train_dataset=data_tensors['train'], eval_dataset=data_tensors['validation'])

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
500,1.663600,1.696680


TrainOutput(global_step=520, training_loss=2.0010773438673755, metrics={'train_runtime': 1596.4722, 'train_samples_per_second': 10.467, 'train_steps_per_second': 0.326, 'total_flos': 8282712592207872.0, 'train_loss': 2.0010773438673755, 'epoch': 4.979054458408139})

In [13]:
trainer.save_model('/kaggle/working/flan_t5_base_tuned')

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1718370023.581ca43f0480.34.0:   0%|          | 0.00/29.7k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

In [21]:
pipe = pipeline('text2text-generation', model='/kaggle/working/flan_t5_base_tuned')
gen_kwargs = {'length_penalty': 0.8, 'num_beams': 8, 'max_length': 128}

ctxt = my_data['test'][93]['context']

query = ctxt
qout = pipe(query, **gen_kwargs)

print(f"Context: {ctxt}", end="\n\n\n")
print(f"Generated: {qout[0]['generated_text']}")

Context: The wraparound process is an intensive, individualized care management process for youths with serious or complex needs. Wraparound was initially developed in the 1980s as a means for maintaining youth with the most serious emotional and behavioral problems in their home and community. During the wraparound process, a team of individuals who are relevant to the well-being of the child or youth (e.g., family members, other natural supports, service providers, and agency representatives) collaboratively develop an individualized plan of care, implement this plan, and evaluate success over time. The wraparound plan typically includes formal services and interventions, together with community services and interpersonal support and assistance provided by friends, kin, and other people drawn from the family's social networks. The team convenes frequently to measure the plan's components against relevant indicators of success. Plan components and strategies are revised when outcomes 